In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import tensorflow as tf

In [48]:
from tensorflow import keras
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Dense

In [9]:
data=pd.read_csv('/content/drive/My Drive/datasets/deep learning/ANN/fake news data.csv')

In [13]:
data.dropna(inplace=True)

In [19]:
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [37]:
x=data.drop(['URLs','Label'],axis=1)
y=data['Label']

In [38]:
messages=x.copy()
messages.reset_index(inplace=True)

In [42]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [44]:
corpus

['four way bob corker skewer donald trump',
 'linklat war veteran comedi speak modern america say star',
 'trump fight corker jeopard legisl agenda',
 'egypt cheiron win tie pemex mexican onshor oil field',
 'jason aldean open snl vega tribut',
 'jetnat fanduel leagu week',
 'kansa tri tax plan similar trump fail',
 'india rbi chief growth import cost inflat newspap',
 'epa chief sign rule clean power plan exit tuesday',
 'talk sale air berlin plane easyjet risk collaps report',
 'u presid donald trump quietli sign law allow warrantless search part va dc md',
 'fantasi footbal team defens rank week',
 'shut play damn basebal',
 'deloitt cyber attack affect client guardian',
 'chuck axe hbd brickyard adam moonlight ace chuck season final',
 'gunman girlfriend said know plan harm',
 'maril danley gunman girlfriend say unawar plan',
 'trump immigr rhetor echo bitter fight',
 'trump bemoan littl appreci san juan mayor beg water',
 'meet militari trump talk calm storm',
 'teacher spark outr

In [45]:
voc_size=10000
onehot=[one_hot(words,voc_size) for words in corpus ]
onehot

[[2225, 1751, 983, 1739, 4163, 1851, 7755],
 [3326, 4107, 7217, 2660, 2510, 2811, 5663, 2021, 1392],
 [7755, 6604, 1739, 5332, 8592, 9717],
 [5514, 4635, 9353, 4454, 3487, 3975, 7665, 5151, 144],
 [5138, 8050, 7387, 2710, 909, 7773],
 [1057, 5178, 5480, 9929],
 [6830, 125, 7875, 9119, 9104, 7755, 8111],
 [7354, 9017, 7496, 8728, 2189, 3322, 5650, 6677],
 [5049, 7496, 2138, 4862, 5677, 1112, 9119, 6026, 9249],
 [3197, 8084, 4375, 8791, 2300, 7070, 4585, 5130, 2083],
 [4009,
  1226,
  1851,
  7755,
  7545,
  2138,
  47,
  1281,
  6099,
  5256,
  1499,
  1537,
  4715,
  6325],
 [2423, 2788, 6072, 6643, 3401, 9929],
 [6189, 3132, 6508, 2161],
 [36, 7542, 2774, 5692, 19, 1599],
 [9464, 5551, 7890, 3735, 2355, 2699, 4727, 9464, 6241, 7118],
 [4312, 6569, 1100, 1676, 9119, 5632],
 [6750, 8276, 4312, 6569, 2021, 7467, 9119],
 [7755, 7788, 3811, 7019, 5570, 6604],
 [7755, 912, 2144, 7842, 8811, 6913, 6916, 7549, 2533],
 [4832, 5815, 7755, 3197, 6797, 4877],
 [3631, 301, 7818, 8535, 2368, 89, 38

In [46]:
sent_size=20
embelded=pad_sequences(onehot,maxlen=sent_size)
embelded

array([[   0,    0,    0, ..., 4163, 1851, 7755],
       [   0,    0,    0, ..., 5663, 2021, 1392],
       [   0,    0,    0, ..., 5332, 8592, 9717],
       ...,
       [   0,    0,    0, ..., 2506, 2840, 6344],
       [   0,    0,    0, ..., 5758, 6548,  184],
       [   0,    0,    0, ..., 9083, 1934, 6230]], dtype=int32)

In [50]:
features=32
model=Sequential()
model.add(Embedding(voc_size,features,input_length=sent_size))
model.add(LSTM(132))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
x_final=np.array(embelded)
y_final=np.array(y)

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [53]:
model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),batch_size=32)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2671 samples, validate on 1317 samples
Epoch 1/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.6484 - accuracy: 0.6103 - val_loss: 0.4871 - val_accuracy: 0.7715
Epoch 2/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.3367 - accuracy: 0.8682 - val_loss: 0.3997 - val_accuracy: 0.8254
Epoch 3/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.1557 - accuracy: 0.9539 - val_loss: 0.4350 - val_accuracy: 0.8337
Epoch 4/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.0730 - accuracy: 0.9779 - val_loss: 0.4921 - val_accuracy: 0.8322
Epoch 5/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.0463 - accuracy: 0.9865 - val_loss: 0.5413 - val_accuracy: 0.8443
Epoch 6/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.0257 - accuracy: 0.9936 - val_loss: 0.6989 - val_accuracy: 0.8421
Epoch 7/50
2671/2671 [==============================] - 3s 1ms/step - loss: 0.0135 - acc

In [54]:
from sklearn.metrics import confusion_matrix
y_pred=model.predict_classes(x_test)
confusion_matrix(y_test,y_pred)

array([[624, 102],
       [108, 483]])

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8405466970387244